In [3]:
# Project setup - paths relative to repository root
import sys
from pathlib import Path

# Set up project paths
PROJECT_ROOT = Path().absolute().parent
sys.path.insert(0, str(PROJECT_ROOT / 'src'))

# Vendor SatSelect (no pip package in the repo)
VENDOR_DIR = PROJECT_ROOT / 'vendor'
SATSELECT_DIR = VENDOR_DIR / 'SatSelect'
sys.path.insert(0, str(VENDOR_DIR))
sys.path.insert(0, str(SATSELECT_DIR))

# Google Earth Engine setup
import ee, geemap
ee.Initialize()

# Import local functions
import functions as fn

# SatSelect wrapper
from SatSelect.wrapper import run_processing

# Bands used by the trained model
dataBands = ['blue', 'green', 'red', 'nir', 're1', 're2', 're3', 'swir1', 'swir2', 'ndvi']
bands = ['blue_neg_diff', 'green_neg_diff', 'red_neg_diff', 'nir_neg_diff',
're1_neg_diff', 're2_neg_diff', 're3_neg_diff', 'swir1_neg_diff',
'swir2_neg_diff', 'ndvi_neg_diff', 'doy_neg_diff', 'blue', 'green',
'red', 'nir', 're1', 're2', 're3', 'swir1', 'swir2', 'ndvi', 'diff_max',
'doy']

# Existing model (loaded from EE asset)
rf = fn.get_trained_model(bands)


In [ ]:
# 1) Define geometry
region = 'Alps'

aoi = (ee.FeatureCollection('projects/ee-simonopravil/assets/LULC/Alps_Carph')
              .filter(ee.Filter.eq('m_range', region))
              )
mask = ee.Image(f'projects/ee-simonopravil/assets/LULC/{region}/RandomForest').select('grassland').gte(5000)
geometry = aoi.geometry()
crs = 'EPSG:3035'


In [5]:
# 2) Create tiles
grid, bounds, transform = fn.create_grid(geometry, crs)
small_grid = bounds.coveringGrid(**{
  'proj':crs, 'scale': 10000
})


In [7]:
# 3) Filter tiles by geometry
filtered_grid = grid.filter(ee.Filter.intersects('.geo', geometry))
tile_ids = filtered_grid.aggregate_array('system:index').getInfo()[0:1]


In [8]:
tile_ids

['39,22']

In [9]:
# 4) Loop through tiles: SatSelect -> features -> predict -> export
for year in [2024, 2023, 2022, 2021, 2020, 2019, 2018]:
    for tile_index, tile_id in enumerate(tile_ids):
        tile = filtered_grid.filter(ee.Filter.eq('system:index', tile_id))
        tile_5 = small_grid.filterBounds(tile)
        geometry = tile.geometry()
        prm = {
                'aoi': geometry,
                'startDate':f'{year}-01-01',
                'endDate': f'{year}-12-31',
                'tileCldFilter': 80,
                's2_score+_cloudBand': 'cs_cdf',
                's2_maskConf': 75,
                'useLandsat': False,
                'l_maskConf': 'Medium',
                'applyMosaic': True,
                'applyHistMatch': False,
                'applyHistMatchBands': ['red', 'green', 'blue', 'nir','swir1', 'swir2'],
                'indices': ['evi', 'ndvi'],
            }
        ic = run_processing(prm).map(lambda i: i.updateMask(mask)).map(fn.addTimeBands)

        # Feature creation
        ic = fn.reduce_quantile(ic, mask, tile_5, 90, crs, 100)
        arrs = []
        for band in dataBands:
            Arr = ic.select(band, 'system:time_start').toArray()
            arrs.append(Arr)
        doyArr = (ic
                    .map(fn.updateCloudMask)
                    .select('doy','system:time_start')
                    .toArray()
                )
        for i in range(0,len(dataBands)):
            ic = ic.map(fn.CreateDiffColl(arrs[i], dataBands[i]))
        ic = ic.map(fn.CreateDiffColl(doyArr, 'doy'))

        for band in dataBands+['doy']:
            ic = fn.time_diff(ic, band)

        ic = ic.map(fn.maxdifference)
        ds = ic.select(bands)
        ds = ds.filterDate(f'{year}-05-01', f'{year}-11-01')

        # Predict using the existing model
        ds_pred = fn.predict_model(ds, rf).select('mowing', 'doy')
        ds_pred_filtered_mode = ds_pred.map(fn.weighted_majority_filter).select('mowing_mode').map(fn.addTimeBands)
        ds_pred_filtered = fn.filter_close_mowing_events(ds_pred_filtered_mode, min_interval_days = 12).select('mowing_filtered', 'doy')

        # Export
        out = (fn.mowing_results(ds_pred_filtered)
                .addBands(ds_pred_filtered.sum())
                .updateMask(mask)
                .unmask(0)
                .updateMask(mask)
                .unmask(999)
                .select(['mowing1DOY', 'mowing2DOY', 'mowing3DOY', 'mowing4DOY', 'mowing5DOY', 'mowing_filtered'],
                        ['mowing1DOY', 'mowing2DOY', 'mowing3DOY', 'mowing4DOY', 'mowing5DOY', 'n_mowings'])
            )
        noDataValue = 0
        exportImage = out.unmask(**{
            'value':noDataValue,
            'sameFootprint': False
        })
        task = ee.batch.Export.image.toDrive(**{
            'image': exportImage.int(),
            'description': f'{region}_Mowing_Intensity{year}_{tile_id}',
            'fileNamePrefix': f'{region}_Mowing_Intensity{year}_{tile_id}',
            'folder':'GrasslandMowing',
            'crs': crs,
            'crsTransform': transform,
            'region': geometry,
            'maxPixels': 1e10
        })
        task.start()
